Imports and Logging Set Up

In [5]:
import os
import logging
import numpy as np
import torch
torch.set_num_threads(2)
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import KFold
from collections import defaultdict
import importlib

import src.dataProcess
import src.models

DATA_DIR = './dataset'

def setup_logging(log_file=None):
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    log_format = '%(asctime)s - [%(levelname)s] - %(message)s'
    if log_file is None:
        logging.basicConfig(level=logging.DEBUG, stream=sys.stdout, format=log_format)
    else:
        logging.basicConfig(level=logging.DEBUG, filename=log_file, filemode='w', format=log_format)

Data Loading

In [7]:
def get_data_loader(dataset, batch_size, k=0, pin_memory=False, save_best=True):
    data_path = os.path.join(DATA_DIR, dataset + '.csv')

    column_names = [
        'top-left', 'top-middle', 'top-right', 'middle-left', 'middle-middle', 'middle-right',
        'bottom-left', 'bottom-middle', 'bottom-right', 'class'
    ]
    # Load data
    data = src.dataProcess.load_data(data_path, column_names)
    
    # Define numerical and categorical features
    categorical_features = column_names[:-1]  # All features except the target
    target_feature = 'class'
    
    # Preprocess data
    preprocessed_data, target_data = src.dataProcess.preprocess_data(data, [], categorical_features, target_feature)
    
    # Split data using KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    train_index, test_index = list(kf.split(data))[k]
    X_train = preprocessed_data[train_index]
    y_train = target_data[train_index]
    X_test = preprocessed_data[test_index]
    y_test = target_data[test_index]
    
    # Create TensorDatasets
    train_set = TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train.astype(np.float32)))
    test_set = TensorDataset(torch.tensor(X_test.astype(np.float32)), torch.tensor(y_test.astype(np.float32)))
    
    # Split train set into train and validation sets
    train_len = int(len(train_set) * 0.95)
    train_sub, valid_set = random_split(train_set, [train_len, len(train_set) - train_len])
    
    if save_best:  # Use validation set for model selection
        train_set = train_sub
    
    # Create DataLoaders
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=pin_memory)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, pin_memory=pin_memory)
    
    return train_loader, valid_loader, test_loader

Training Methods

In [8]:
def train_model(args):
    setup_logging(args.log)
    torch.manual_seed(42)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    writer = SummaryWriter(args.folder_path) if args.rank == 0 else None

    dataset = args.data_set
    train_loader, valid_loader, test_loader = get_data_loader(dataset, args.batch_size, k=args.ith_kfold, pin_memory=True, save_best=args.save_best)

    # Define model dimensions
    discrete_flen = len(train_loader.dataset[0][0])  # Adjust based on your dataset
    continuous_flen = 0  # Adjust based on your dataset
    dim_list = [(discrete_flen, continuous_flen)] + list(map(int, args.structure.split('@'))) + [1]  # Adjust based on your dataset

    rrl = RRL(dim_list=dim_list,
              device_id=device,
              use_not=args.use_not,
              is_rank0=args.rank == 0,
              log_file=args.log,
              writer=writer,
              save_best=args.save_best,
              estimated_grad=args.estimated_grad,
              use_skip=args.skip,
              save_path=args.model,
              use_nlaf=args.nlaf,
              alpha=args.alpha,
              beta=args.beta,
              gamma=args.gamma,
              temperature=args.temp)

    rrl.train_model(
        data_loader=train_loader,
        valid_loader=valid_loader,
        lr=args.learning_rate,
        epoch=args.epoch,
        lr_decay_rate=args.lr_decay_rate,
        lr_decay_epoch=args.lr_decay_epoch,
        weight_decay=args.weight_decay,
        log_iter=args.log_iter)

Model Loading

In [9]:
def load_model(path, device, log_file=None):
    checkpoint = torch.load(path, map_location='cpu')
    saved_args = checkpoint['rrl_args']
    rrl = RRL(
        dim_list=saved_args['dim_list'],
        device_id=device,
        is_rank0=True,
        use_not=saved_args['use_not'],
        log_file=log_file,
        estimated_grad=saved_args['estimated_grad'],
        use_skip=saved_args['use_skip'],
        use_nlaf=saved_args['use_nlaf'],
        alpha=saved_args['alpha'],
        beta=saved_args['beta'],
        gamma=saved_args['gamma'])
    rrl.net.load_state_dict(checkpoint['model_state_dict'])
    return rrl

Test Function

In [11]:
def test_model(args):
    rrl = load_model(args.model, args.device, log_file=args.test_res)
    dataset = args.data_set
    train_loader, valid_loader, test_loader = get_data_loader(dataset, args.batch_size, args.ith_kfold, save_best=False)
    rrl.test(test_loader=test_loader, set_name='Test')
    if args.print_rule:
        with open(args.rrl_file, 'w') as rrl_file:
            rule2weights = rrl.rule_print(db_enc.X_fname, db_enc.y_fname, train_loader, file=rrl_file, mean=db_enc.mean, std=db_enc.std)
    else:
        rule2weights = rrl.rule_print(db_enc.X_fname, db_enc.y_fname, train_loader, mean=db_enc.mean, std=db_enc.std, display=False)
    
    metric = 'Log(#Edges)'
    edge_cnt = 0
    connected_rid = defaultdict(lambda: set())
    ln = len(rrl.net.layer_list) - 1
    for rid, w in rule2weights:
        connected_rid[ln - abs(rid[0])].add(rid[1])
    while ln > 1:
        ln -= 1
        layer = rrl.net.layer_list[ln]
        for r in connected_rid[ln]:
            con_len = len(layer.rule_list[0])
            if r >= con_len:
                opt_id = 1
                r -= con_len
            else:
                opt_id = 0
            rule = layer.rule_list[opt_id][r]
            edge_cnt += len(rule)
            for rid in rule:
                connected_rid[ln - abs(rid[0])].add(rid[1])
    logging.info('\n\t{} of RRL  Model: {}'.format(metric, np.log(edge_cnt)))

Main Function

In [12]:
importlib.reload(src.dataProcess)
from src.dataProcess import *
importlib.reload(src.models)
from src.models import *

def main(args):
    train_model(args)
    test_model(args)

class Args:
    data_set = 'tictactoe/tic-tac-toe'
    batch_size = 32
    ith_kfold = 0
    save_best = True
    folder_path = './logs'
    rank = 0
    use_not = False
    log = None
    structure = '10@10@10'
    learning_rate = 0.01
    epoch = 50
    lr_decay_rate = 0.75
    lr_decay_epoch = 100
    weight_decay = 0.0
    log_iter = 50
    model = './model.pth'
    test_res = './test_results.log'
    print_rule = True
    rrl_file = './rules.txt'
    alpha = 0.999
    beta = 8
    gamma = 1
    temp = 0.01
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    estimated_grad = True
    skip = False

args = Args()
main(args)

FileNotFoundError: [Errno 2] No such file or directory: './dataset/your_dataset.data'